# C/C++ __stdcall DLL, Excel VBA

* Change the current working directory into `./demo`

In [2]:
%cd demo

J:\SEU\SEE\PySEE\home\notebook\demo


In [2]:
%pwd

'F:\\SEU\\SEE\\PySEE\\home\\notebook\\demo'

## 1 C/C++ __stdcall DLL 

  For **Visual Basic applications** (or applications in other languages such as Pascal or Fortran) to call functions in a C/C++ DLL, the functions must be exported using the correct calling convention without any name decoration done by the compiler.
  
VBA can **only** call **`__stdcall`** functions, not `__cdecl` functions. 
  
*  **`__stdcall`** creates the correct calling convention for the function (the called function cleans up the stack and parameters are passed from **right to left(从右向左)**

*  **`__declspec(dllexport)`** is used on an `exported` function in a DLL

For Example:

```c
__declspec(dllexport) double __stdcall  mean(double data[], int size);;
```



Below is the example of techniques which facilitate the use of use of MinGW to create DLLs, exporting functions which may be called from Visual Basic Application with Excel. 

**Example:** The Saturation-Pressure Equation
  
    
  pSat(double t);
  



### 1.1 Create DLL with `__stdcall` calling convention


**The ·MANY· headers**

When you create header files for your `__stdcall` DLLs,

* ` __declspec(dllexport) ` adds the `export` directive to the object fileworks

* ` __declspec(dllimport)`  on the declarations of the public symbols


1.The headers for building the Windows **`__stdcall`** shared library  

```c
__declspec(dllexport) double __stdcall pSat(double t);

```
2.The header for building an executable that uses the  shared library.

```c
__declspec(dllimport) double __stdcall pSat(double t);
```


**The Multi-purpose header**

Create a DLL of IAPWS-IF97-Region4 with the following code:

* R134aSat.h

* R134aSat.c

The following header `region4.h`  declares the interface for 

* building the **Windows/Linux** shared library  

* building an executable that uses the  shared library.

Using the macro **PORT** as the return type of function 
 
 
* For Windows,**export** all functions as `__stdcall`.

```c
#define PORT __declspec(dllexport) double __stdcall
```

* For Windows,**import** all functions as `__stdcall`.

```c
#define PORT __declspec(dllimport) double __stdcall
```

* For Linux as  `__cdecl`

```c
#define PORT double
```

**#pragma once** 

This is a `non-standard` but widely supported preprocessor directive designed to cause the current source file to be included **only once** in a single compilation. 

**extern "C"** 

extern "C" syntax to allow the run-time library functions to be used in C++ programs.*

In [8]:
%%file ./include/R134aSat.h
#pragma once

#ifdef __cplusplus
extern "C" {
#endif

#ifdef BUILD_DLL

    #ifdef WIN32
        #define PORT __declspec(dllexport) double __stdcall 
    #else
        #define PORT double
    #endif    

#else

    #ifdef WIN32
        #define PORT __declspec(dllimport) double __stdcall   
    #else
        #define PORT double
    #endif    

#endif

PORT pSat(double t);
        
#ifdef __cplusplus
	}
#endif        

Overwriting ./include/R134aSat.h


In [15]:
%%file ./src/R134aSat.c
#include <math.h>
#include "R134aSat.h"

PORT pSat(double t)
{
	const double ti[]={1,1.5,2.3,3.6,5.2,7.3};
    const double Ni[]={-7.6896400207782598, 2.0859760566425463, -2.6755347075503888,
                       0.3010493765467589, -5.8583601582059233, 3.4788072104059631};
    double Pcrit=4059280.0;
    double Tcrit=374.21;
    double Treduce=374.18;
    double summer=0,theta;
    double T=t+273.15;
    theta=1.0-T/Treduce;
    for (int i=0;i<=5;i++)
    {
        summer +=Ni[i]*pow(theta,ti[i]);
    }
    return  Pcrit*exp(summer*Tcrit/T)/1.0e6;
}

Overwriting ./src/R134aSat.c


In [48]:
!gcc -c -DBUILD_DLL -o ./obj/R134aSat.o ./src/R134aSat.c -I./include
!gcc -shared -o ./bin/libR134aSat.dll ./obj/R134aSat.o  -Wl,--add-stdcall-alias

C:/TDM-GCC-64/bin/../lib/gcc/x86_64-w64-mingw32/9.2.0/../../../../x86_64-w64-mingw32/bin/ld.exe: cannot open output file ./bin/libR134aSat.dll: Permission denied
collect2.exe: error: ld returned 1 exit status


* -DBUILD_DLL:
   
  * -Dname: `Predefine name as a macro`, with definition 
  

*  -Wl,option 

   Pass **option** as an option to the **linker**. If option contains commas, it is split into multiple options at the commas.


* --add-stdcall-alias:
   
   This adds an **undecorated** alias for the `exported function names` that is simply **the name of the function** 

### 1.2  Add -static-libgcc ,-output-def=libregion4.def

 Links the GNU `libgcc` library `statically`

In [13]:
!gcc -c -DBUILD_DLL -o ./obj/R134aSat.o ./src/R134aSat.c -I./include
!gcc -shared -o ./bin/libR134aSat.dll -static-libgcc ./obj/R134aSat.o -Wl,--add-stdcall-alias,-output-def=./bin/libR134aSat.def

**-static-libgcc**

* This option links the GNU `libgcc` library **statically** 
   
**-output-def=libregion4.def**

* Name of `.def` file to be created.
    
* **def:** A module-definition  file is a text file containing one or more module statements that describe various attributes of a DLL

In [ ]:
%load ./bin/libR134aSat.def


In [16]:
%%file ./makefile-R134aSat.mk

CC=gcc
CFLAGS=-DBUILD_DLL

SRCDIR= ./src/
OBJDIR= ./obj/
BINDIR= ./bin/
INCDIR=./include/

all: libregion4

libregion4: obj
	 $(CC) -shared -o $(BINDIR)libR134aSat.dll -static-libgcc $(OBJDIR)R134aSat.o -Wl,--add-stdcall-alias,-output-def=libR134aSat.def
obj: 
	 $(CC) -c $(CFLAGS) -o $(OBJDIR)R134aSat.o $(SRCDIR)R134aSat.c -I$(INCDIR)


Writing ./makefile-R134aSat.mk


In [17]:
!make -f makefile-R134aSat.mk

gcc -shared -o ./bin/libR134aSat.dll -static-libgcc ./obj/R134aSat.o -Wl,--add-stdcall-alias,-output-def=libR134aSat.def


### 1.3 Call Dll from C/C++

In [44]:
%%file ./src/mainR134aSat.c

#include <stdio.h> 
#include "R134aSat.h"

int main() {
     double t=0;
     printf("Temp: %f, Saturation P(MPa) is %f\n",t, pSat(t));  
     return 0;
}

Overwriting ./src/mainR134aSat.c


In [45]:
!gcc -c -o ./obj/mainR134aSat.o ./src/mainR134aSat.c -I./include/
!gcc -o  ./bin/mainR134aSat ./obj/mainR134aSat.o  -L./bin/ -lR134aSat

In [46]:
!.\bin\mainR134aSat

Temp: 0.000000, Saturation P(MPa) is 0.292968


### 1.4 Python API

* `__stdcall` calling convention: **windll, WINFUNCTYPE**

In [14]:
%%file ./src/R134aSat.py

from ctypes import windll,c_double,WINFUNCTYPE

flib = windll.LoadLibrary('./bin/libR134aSat.dll')
prototype = WINFUNCTYPE(c_double,c_double)

def pSat(t):
    f = prototype(("pSat", flib),)
    return f(t)

Overwriting ./src/R134aSat.py


**add `R134aSat.py` into the interperte search path**

In [31]:
import sys
sys.path.append('./src')

In [32]:
from R134aSat import *
t=0
p=pSat(t)
print(p)

0.29296756119458117


In [12]:
%%file ./src/testR134aSat.py

import unittest
from  R134aSat import *

class  R134aSatest (unittest.TestCase):

    def setUp(self):
        #  t, p
        self.table=[[-8, 0.21704],
                    [0, 0.29282],
                    [8,0.38756]
                   ]
                    
      
    def test_pSat(self):
        places = 3
        for item in  self.table:
             self.assertAlmostEqual(pSat(item[0]),item[1],places)

if __name__ == '__main__':
    unittest.main()       

Overwriting ./src/testR134aSat.py


In [13]:
%run ./src/testR134aSat.py

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


## 2 Call DLL from Excel VBA(64bits)

Do as the following steps:

### 2.1 Put  DLL in the default path of Windows'dll 


`libR134aSat.dll`  in `C:\windows\system`


### 2.2 Create the xlsm

`demo-R134aSat.xlsm` in `./demo/ExcelVBA/`


### 2.3 VBA module to call the DLL.

There are a few ways to open up the **VBA Editor** in Excel.

* From the `Developer Tools` tab, you can click the `Visual Basic` button.

* A keyboard shortcut: press **"Alt+F11"** :

![vba](./img/vba.jpg)

Into **VBA** Editor, then， create the module **mathfuns** to call library:

```VBA
Declare PtrSafe Function pSat Lib "libtR134aSat" (ByVal t As Double) As Double

Public Function CalpSat(ByVal t As Double) As Double
    CalpSat = pSat(T)
End Function
 
```

### 2.4  call VBA methods in cells

![R134aVBA](./img/vcr/R134aVBA.jpg)

## Reference

**Windows's DLL**

* [What is a DLL and How Do I Create or Use One?](http://www.mingw.org/wiki/DLL)

* DLLs in Visual C++ https://msdn.microsoft.com/en-us/library/1ez7dh12.aspx

    
**Microsoft Excel**

* Excel VBA Programming:  http://www.homeandlearn.org/the_excel_vba_editor.html

* [Language reference for Visual Basic for Applications(VBA)](https://docs.microsoft.com/en-us/office/vba/api/overview/language-reference)

* [Excel VBA reference](https://docs.microsoft.com/en-us/office/vba/api/overview/excel)

* Calling DLL Functions from Visual Basic Applications https://msdn.microsoft.com/en-us/library/dt232c9t.aspx

**Excel4Engineering**

Examples of Excel VBA for Engineering: Analysizing the Ideal Rankine Cycle, Monitoring Industrial Process 

* https://github.com/thermalogic/Excel4Engineering
    
